# TP1 - Branch & Bound/Cuts for the TSP

## The `minilp` python package

The `minilp` python package is a small python package that allows us to easily model (integer) linear program. The package comes with an interface to common linear programming solvers (`cplex`, `scipy`) but no integer linear programming solver.

In [1]:
from minilp import *

lp = problem('My first LP problem')

# create two continuous variables within [0, 4]
x1, x2 = lp.continuous_var_list(2, 0, 4)

# add constraints
lp.add_constraint(-3 * x1 + 4 * x2 <= 7)
lp.add_constraint(2 * x2 <= 5)
lp.add_constraint(6 * x1 + 4 * x2 <= 25)
lp.add_constraint(2 * x1 - x2 <= 6)

# set the objective function
lp.set_objective(x1 + 2 * x2, sense=max)

# solve the problem
res = lp.lp_solve()
print(res)
print('x1 = {:.4f}, x2 = {:.4f}'.format(res.get_value(x1), res.get_value(x2)))

status = optimal, obj. = 7.5
x1 = 2.5000, x2 = 2.5000


The `minilp` package allows you to modelise simple integer linear programs:

In [2]:
from minilp import *

lp = problem('My first LP problem')

# create two integer variables within [0, 4]
x1, x2 = lp.integer_var_list(2, 0, 4)

# add constraints
lp.add_constraint(-3 * x1 + 4 * x2 <= 7)
lp.add_constraint(2 * x2 <= 5)
lp.add_constraint(6 * x1 + 4 * x2 <= 25)
lp.add_constraint(2 * x1 - x2 <= 6)

# set the objective function
lp.set_objective(x1 + 2 * x2, sense=max)

But it does not provide a integer linear program solver &mdash; the `lp_solve` method will always solve the linear relaxation of the problem:

In [3]:
res = lp.lp_solve()
print(res)
print('x1 = {:.4f}, x2 = {:.4f}'.format(res.get_value(x1), res.get_value(x2)))

status = optimal, obj. = 7.5
x1 = 2.5000, x2 = 2.5000


The `minilp` package allows you to modelise `<=`, `>=` or `==` (in)equalities. You can create linear expression by simply adding, substracting or multiplying values (`int` or `float`) and variables or existing expressions. You can use the standard python `sum` to sum a bunch of expressions or variables, and the `minilp.dot` function to compute the dot product of two vectors.

**Exercice:** Complete the following code to create a simple model for the knapsack problem.

In [19]:
from minilp import *

p = [1, 4, 5, 3, 5] # profits
w = [3, 4, 3, 5, 9] # weights
k = 10 # capacity

# A simple knapsack
kp = problem('Simple knapsack')

# TODO: Create variables, add constraints and set the objective.
x = kp.binary_var_list(len(p))
#x = lp.integer_var_list(len(p), 0, 1)
kp.add_constraint(dot(x, w) <= k)

kp.set_objective(dot(x, p), sense = max)

# We can solve the linear relaxation:
res = kp.lp_solve()
print(res)
print(res.get_values(x))

status = optimal, obj. = 10.8
[0, 1.0, 1.0, 0.6000000000000001, 0]


In [17]:
help(res)

Help on result in module minilp.result object:

class result(builtins.object)
 |  Methods defined here:
 |  
 |  __bool__(self)
 |  
 |  __init__(self, success=False, status='unknown', objective=nan, variables=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  get_value(self, var)
 |      Retrieve the value associated to the given variable.
 |      
 |      Parameters:
 |        - var A minilp.expr.var object.
 |      
 |      Return: Value associated with the given variable.
 |  
 |  get_values(self, *args)
 |      Return values associated to the given variables.
 |      
 |      Parameters:
 |        - vs Iterable of minilp.expr.var.
 |      
 |      Return: List of value associated with the variables.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __w

## Generic Branch & Bound

**Exercice:** Implement a generic branch and bound algorithm that can be used to solve any `minilp.problem` instance.

**Tips:** Do not hesitate to add (some) logging messages to your algorithm in order to visualize the progression.

**Tips:** You can add and remove constraint(s) using the `add_constraint(s)` and `del_constraint(s)` method of `minilp.problem`.

In [12]:
p = problem()

x, y = p.integer_var_list(2)

type(x + y <= 8)

minilp.expr.cons

In [28]:
help(minilp.result)

Help on module minilp.result in minilp:

NAME
    minilp.result - # -*- encoding: utf-8 -*-

CLASSES
    builtins.object
        result
    
    class result(builtins.object)
     |  Methods defined here:
     |  
     |  __bool__(self)
     |  
     |  __init__(self, success=False, status='unknown', objective=nan, variables=None)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  __repr__(self)
     |      Return repr(self).
     |  
     |  get_value(self, var)
     |      Retrieve the value associated to the given variable.
     |      
     |      Parameters:
     |        - var A minilp.expr.var object.
     |      
     |      Return: Value associated with the given variable.
     |  
     |  get_values(self, *args)
     |      Return values associated to the given variables.
     |      
     |      Parameters:
     |        - vs Iterable of minilp.expr.var.
     |      
     |      Return: List of value associated with the variables.
   

In [74]:
import datetime as dt
import math
import numpy as np

from minilp import *
import minilp.result


class branch_and_bound:
    
    # Number of iterations between to log (not including exceptional log).
    LOG_ITERATIONS = 5
    
    def __init__(self, pb, lp_solver=solvers.get_default_solver()):
        """ Create a new branch_and_bound algorithm for the given 
        problem with the given solver.
        
        Parameters:
          - pb Problem to solve (instance of minilp.problem).
          - lp_solver Solver to use to solve linear relaxation.
        """        
        
        # Instance of minilp.problem to solve.
        self.problem = pb
        
        # Current best solution found (default constructor create
        # an "unknown" result).
        self.current_best = minilp.result.result()
        
        # List of nodes of the algorithm.
        self.nodes = []
        
        # Solver for solving relaxation.
        self.lp_solver = lp_solver
        
        # Iteration counter.
        self.nb_iterations = 0
                
    def insert_first_node(self):
        """ Insert the first node in the list of nodes. """
        self.nodes.append( [] )
    
    def get_next_node(self):
        """ Retrieve the next node to evaluate.
        
        Return: The next node to evaluate by the algorithm. """
        return self.nodes.pop(0)
            
    def is_done(self):
        """ Check if the algorithm has finished.
        
        Return: True if the algorithm should stop, False otherwise. """
        return len(self.nodes) == 0

    def get_first_non_integral(self, sol):
        """ Retrieve the first integer variable in the underlying problem 
        whose value is not integral in the given solution.
        
        Parameters:
          - sol Solution of the problem.
    
        Return: The first variable (minilp.var) whose value is not integral, 
        or None if no such variable exists.
        """
    
        # Hint: help(p), help(s) to obtain documentation about the class problem
        # and solution.
        for v in self.problem.variables:
            if v.category == int:
                value = sol.get_value(v)
                if abs(round(value) - value) > 1e-6:
                    return v
        
        return None
    
    def compare_solution(self, l, r):
        """ Compare the two given solutions, returning True if the left
        one is better than the right one for the current problem.
        
        Parameters:
          - l, r The two solutions (minilp.result) to compare.
          
        Return: True if the left solution is better than the right one, or
        if the right solution has no solution.
        """
        if np.isnan(r.objective):
            return True
        if self.problem.sense == min:
            return l.objective < r.objective
        return l.objective > r.objective
    
    def log_solution(self, res, new_best=False):
        """ Log the given solution with time information.
        
        Parameters:
          - res Solution (minilp.result) to log.
          - new_best Indicates if this solution is new best solution (integer).
        """
        print('{} {:5d} {:5d} {:9g}{}'.format(
            dt.datetime.now().strftime('%T'), 
            self.nb_iterations, len(self.nodes), res.objective,
            '*' if new_best else ''))
        
    def iterate(self):
        """ Perform an iteration of the branch-and-bound algorithm. """
        
        # Increment counter and log.
        if self.nb_iterations % branch_and_bound.LOG_ITERATIONS == 0:
            self.log_solution(self.current_best)
            
        self.nb_iterations += 1
        
        # Retrieve the next node to handle:
        node = self.get_next_node()
        
        # Hint: You can use add_constraint and del_constraint on the
        # underlying problem (self.problem).
        
        # TODO
        self.problem.add_constraints(node)
        sol = self.problem.lp_solve(solver = self.lp_solver)
        self.problem.del_constraints(node)

        if sol and self.compare_solution(sol, self.current_best):
            non_integral = self.get_first_non_integral(sol)
        
            # Save if we found a solution and it's better than current:
            if non_integral is None :
                self.current_best = sol
            else :
                seuil = int(sol.get_value(non_integral))
                self.nodes.append( node + [ non_integral <= seuil ] )
                self.nodes.append( node + [ non_integral >= seuil + 1 ] )
            
            
    def run(self):
        """ Run the branch-and-bound algorithm and return a minilp.result. 
        
        Return: A minilp.result containing the result of running the branch-and-bound
        algorithm. """
        
        print('B&B using {} to solve linear relaxation'.format(self.lp_solver.__class__.__name__))
        
        # Insert the first node in the list:
        self.insert_first_node()
        while not self.is_done():
            self.iterate()
                
        # Return the best solution found (if any).
        return self.current_best

**Exercice:** Test your algorithm on the knapsack problem above.

In [75]:
our_solver = branch_and_bound(kp)
s = our_solver.run()
print(s.objective)
print(s.get_values(kp.variables))

B&B using docplex to solve linear relaxation
11:48:52     0     1       nan
10.0
[1.0, 1.0, 1.0, 0, 0]


**Exercice:** Create other instances of the knapsack problem to reach the "limits" of your implementation &mdash; What is the largest instance you can solve in e.g. less than 5 seconds?

## The Travelling Salesman Problemn (TSP)


Given a list of $n$ cities and the distances $c_{ij}$ between each pair of cities, you want to find the shortest circuit that visits each city **exactly once** and comes back to the first visited city.

The goal of this section is to implement a generic branch & bound algorithm and a branch & cuts algorithm for the travelling salesman problem, using the `minilp` python package.

### Creating a model for the TSP

**Exercice:** Create the `tsp_relax` function and either the `tsp_mtz` or `tsp_flow` (or both) function(s):

- `tsp_relax(distances)` &mdash; Create a `minilp.problem` object representing a TSP instance without subtour constraints using the given matrix of distances. 

- `tsp_mtz(distances)` &mdash; Create a MTZ formulation for the TSP using the given matrix of distances.

- `tsp_flow(distances)` &mdash; Create a flow formulation for the TSP using the given matrix of distances.

In [79]:
from minilp import *
import numpy as np

def tsp_relax(distances):
    tsp = problem("Travelling Salesman Problem Relax")
    
    n = len(distances)
    x = np.array(tsp.binary_var_list(n * n)).reshape((n, n))
    
    tsp.set_objective((x * np.array(distances)).sum(), sense = min)
    
    return tsp, x

def tsp_mtz(distances):
    tsp = problem("Travelling Salesman Problem MTZ")
    
    n = len(distances)
    x = np.array(tsp.binary_var_list(n * n)).reshape((n, n))
    
    tsp.set_objective((x * np.array(distances)).sum(), sense = min)
    
    return tsp, x

def tsp_flow(distances):
    pass

In [82]:
import tsp.data

distances = tsp.data.grid6
tsp, x = tsp_relax(distances)

print(tsp.objective)

=== MTZ ===
32 * _x1 + 47 * _x2 + 49 * _x3 + 16 * _x4 + 44 * _x5 + 32 * _x6 + 41 * _x8 + 22 * _x9 + 31 * _x10 + 10 * _x11 + 47 * _x12 + 41 * _x13 + 15 * _x15 + 18 * _x16 + 40 * _x17 + 49 * _x18 + 22 * _x19 + 15 * _x20 + 24 * _x22 + 36 * _x23 + 16 * _x24 + 31 * _x25 + 18 * _x26 + 24 * _x27 + 9 * _x29 + 44 * _x30 + 10 * _x31 + 40 * _x32 + 36 * _x33 + 9 * _x34


The `tsp.data` packages contains grid of distances of various sizes (5, 6, 7, 8, 9, 10, 15, 17, 26, 42).

**Exercice:** Using the `tsp_mtz` and `banb` functions you implemented, solve the **small** TSP instances found in `tsp.data`.

**Question:** How large are the instances you are able to solve in a reasonable amount of time?

In [ ]:
from minilp import *
import tsp.data

distances = tsp.data.grid6

print('=== MTZ ===')
tsp, (x, u) = tsp_mtz(distances)
r = branch_and_bound(tsp, solvers.docplex()).run()
print(r)

print('\n=== Flow ===')
tsp, (x, y) = tsp_flow(distances)
r = branch_and_bound(tsp, solvers.docplex()).run()
print(r)

### Creating a branch & cuts for the TSP

**Exercice:** Create the function `tsp_cuts` that implements a branch & cuts for the TSP using the `banb` algorithm you implemented.

In [ ]:
class tsp_branch_and_cuts:
    
    def __init__(self, distances, ilp_solver=branch_and_bound):
        """ Create a new TSP branch-and-cuts algorithm for the given
        instance and solver.
        
        Parameters:
          - distances Distances for the algorithm (a NxN grid of distances).
          - ilp_solver Solver class or function returning a ILP solver.
        """
        
        # Number of nodes in the instance.
        self.n = len(distances)
        
        # TSP problem and variables X from the relaxation.
        self.tsp, self.x = tsp_relax(distances)
        
        # ILP solver.
        self.ilp_solver = ilp_solver
        
    def get_subtours(self, sol):
        """ Retrieve the list of subtours in the given solution for the 
        underlying TSP problem.
        
        Parameters:
          - sol Solution where subtours should be found.
          
        Return: A list of subtours, each subtour being a list of node
        numbers (e.g., [1, 2, 3] is a subtour containing node 1, 2 and 3.)
        """ 
        return None
        
    def add_subtour_constraint(self, subtour):
        """ Add a constraint for the given subtour to the underlying TSP problem.
        
        Parameters:
          - subtour Subtour for which a constraint should be added.
        """
        pass

        
    def run(self):
        """ Run this branch-and-cuts algorithm on the underlying problem. """
        
        # Current solution.
        sol = minilp.result.result()
        
        while True:
            pass
                
        return sol


**Exercice:** Using the `tsp_cuts` functions, solve TSP instances found in `tsp.data`.

**Question:** How large are the instances you are able to solve in a reasonable amount of time?

In [ ]:
import tsp.data

distances = tsp.data.grid5

print('=== Cuts ===')
r = tsp_branch_and_cuts(distances, lambda p: branch_and_bound(p, solvers.docplex())).run()
print(r)